# Examples for using methods from `process_util`

## Parallel Processing

In [ ]:
from tqdm import tqdm_notebook

In [ ]:
from process_util import parallel_iwork_unordered, parallel_work

In [ ]:
from multiprocessing import current_process
import random
import time

def worker(values):
    # Count how much work each worker does.
    count = 0
    for value in values:
        time.sleep(random.uniform(0.05, 0.1))
        count += 1
        yield (current_process().name, count, value)

In [ ]:
inputs = ["hey", "world", "look", "at", "me"]

In [ ]:
outputs_iter = parallel_iwork_unordered(
    worker, inputs, process_count=3)
outputs = list(tqdm_notebook(outputs_iter, total=len(inputs)))

assert set(output[2] for output in outputs) == set(inputs)  # Unordered
outputs

In [ ]:
# Non-parallel
outputs_iter = parallel_iwork_unordered(
    worker, inputs, process_count=0)
outputs = list(tqdm_notebook(outputs_iter, total=len(inputs)))

assert [output[2] for output in outputs] == inputs
outputs

In [ ]:
outputs = parallel_work(
    worker, inputs, process_count=3, progress_cls=tqdm_notebook)

assert [output[2] for output in outputs] == inputs
outputs

In [ ]:
# No progress bar.
outputs = parallel_work(worker, inputs, process_count=0)

assert [output[2] for output in outputs] == inputs
outputs

# Process Manipulation

## CapturedProcessGroup (high level)

In [ ]:
from contextlib import closing
from process_util import CapturedProcessGroup

In [ ]:
# WARNING: With Python 2, newlines may not always be correctly processed...
procs = CapturedProcessGroup()
with closing(procs):
    procs.add("bash", ["bash", "-c", "for i in $(seq 3); do echo Hello ${i}; sleep 0.2; done"])
    while procs.poll() == {}:
        pass
assert procs.poll() == {"bash": 0}

# CapturedProcess (lower-level)

In [ ]:
from functools import partial
from subprocess import Popen
import time

from process_util import (
    on_context_exit,
    signal_processes,
    CapturedProcess,
    CapturedProcessGroup,
    print_prefixed,
)

In [ ]:
# Signaling.
p = Popen(["sleep", "10"])
with on_context_exit(lambda: signal_processes([p])):
    time.sleep(1.)
assert p.returncode == -2

In [ ]:
args = ["bash", "-c", "while :; do echo Hello; read; done"]
p = CapturedProcess(
    args, on_new_text=partial(print_prefixed, prefix=" | "))
with p.scope:
    for i in xrange(3):
        while "Hello" not in p.output.get_text():
            time.sleep(0.01)
            p.poll()
        p.output.clear()
        p.proc.stdin.write("\n")